The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 4: Regular Batch Inference

In Stage 3: Deploy the Model in Wallaroo, the housing model created and tested in Stage 2: Training Process Automation Setup was uploaded to a Wallaroo instance and added to the pipeline `housing-pipe` in the workspace `housepricing`.  This pipeline can be deployed at any point and time and used with new inferences.

For the purposes of this demo, let's say that every month we find the newly entered and still-unsold houses and predict their sale price.

The predictions are entered into a staging table, for further inspection before being joined to the primary housing data table.

We show this as a notebook, but this can also be scripted and scheduled, using CRON or some other process.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

This process will use the following steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and the `housepricing` workspace.
* [Deploy the Pipeline](#deploy-the-pipeline): Deploy the pipeline to prepare it to run inferences.
* [Read In New House Listings](#read-in-new-house-listings): Read in the previous month's house listings and submit them to the pipeline for inference.
* [Send Predictions to Results Staging Table](#send-predictions-to-results-staging-table): Add the inference results to the results staging table.

### Connect to Wallaroo

Connect to the Wallaroo instance and set the `housepricing` workspace as the current workspace.

In [1]:
import json
import pickle
import wallaroo
import pandas as pd
import numpy as np
import pyarrow as pa
import datetime

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo_client import get_workspace

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [3]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

In [4]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

In [5]:
new_workspace = get_workspace(workspace_name, wl)
_ = wl.set_current_workspace(new_workspace)

### Deploy the Pipeline

Deploy the `housing-pipe` workspace established in Stage 3: Deploy the Model in Wallaroo (`03_deploy_model.ipynb`).

In [6]:
pipeline = new_workspace.pipelines()[0]
pipeline

name,housing-pipe
created,2024-03-06 22:04:07.428588+00:00
last_updated,2024-03-06 22:30:05.853639+00:00
deployed,False
arch,None
accel,None
tags,
versions,"8ee88c49-3d09-43b6-b3a3-f1fd7010d9ba, 99204f60-a7fc-473c-881e-e17a8d512774, 8322e7d7-d5ce-417e-a626-6dc4ba327b11, 3095d26d-d7f8-4478-ac2f-92df819ceb14, 7b0e4d58-2232-4a32-8757-5152ffb0d6ef, 21952c2b-89ef-45a3-b5c0-1685af58fa10, 1e6ac00f-9e81-48c6-9956-3a5b27e44616, 0f676ed7-9cd3-4021-82d4-8b180aa4fb00, 9729b5fa-2184-4b0c-8c72-906e5871c276, 895dac3a-fbfc-462c-856c-9ad47a946bf2, a4ffd306-70f6-4668-a236-b90522f0fdcb, 374fb9d1-afd1-47bb-a57e-fd6e3134ded2, ab03935a-38e2-41c3-b05b-bb0cde7088c7, 8f2dc40b-f190-4949-9568-021649630d3f, 3af8a7e3-b817-4031-ac68-c6dd09cbad95, b0524141-5a52-4824-b959-d26892699171"
steps,housepricemodel
published,False


In [8]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,housing-pipe
created,2024-03-06 22:04:07.428588+00:00
last_updated,2024-03-06 22:33:25.001128+00:00
deployed,True
arch,None
accel,None
tags,
versions,"bd31aef3-3c73-4315-b11c-541034a96b32, 8ee88c49-3d09-43b6-b3a3-f1fd7010d9ba, 99204f60-a7fc-473c-881e-e17a8d512774, 8322e7d7-d5ce-417e-a626-6dc4ba327b11, 3095d26d-d7f8-4478-ac2f-92df819ceb14, 7b0e4d58-2232-4a32-8757-5152ffb0d6ef, 21952c2b-89ef-45a3-b5c0-1685af58fa10, 1e6ac00f-9e81-48c6-9956-3a5b27e44616, 0f676ed7-9cd3-4021-82d4-8b180aa4fb00, 9729b5fa-2184-4b0c-8c72-906e5871c276, 895dac3a-fbfc-462c-856c-9ad47a946bf2, a4ffd306-70f6-4668-a236-b90522f0fdcb, 374fb9d1-afd1-47bb-a57e-fd6e3134ded2, ab03935a-38e2-41c3-b05b-bb0cde7088c7, 8f2dc40b-f190-4949-9568-021649630d3f, 3af8a7e3-b817-4031-ac68-c6dd09cbad95, b0524141-5a52-4824-b959-d26892699171"
steps,housepricemodel
published,False


### Read In New House Listings

From the data store, load the previous month's house listing, prepare it as a DataFrame, then submit it for inferencing.

In [9]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} where date > DATE(DATE(), '-1 month') AND sale_price is NULL"
print(query)

# read in the data
# can't have null values - turn them into 0
newbatch = pd.read_sql_query(query, conn)
newbatch['sale_price'] = newbatch.sale_price.apply(lambda x: 0)
display(newbatch.shape)
display(newbatch.head(10).loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]])

select * from house_listings where date > DATE(DATE(), '-1 month') AND sale_price is NULL


(843, 22)

,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot
0,1695900060,2024-02-19,535000.0,4,1.00,1610,2982
1,1432900240,2024-02-16,205000.0,3,1.00,1610,8579
2,7960900060,2024-02-12,2900000.0,4,3.25,5050,20100
3,6378500125,2024-02-09,436000.0,2,1.00,1040,7538
4,2022069200,2024-02-13,455000.0,4,2.50,2210,49375
5,9412900055,2024-02-13,405000.0,3,1.75,2390,6000
6,7424700045,2024-02-21,2050000.0,5,3.00,3830,8480
7,3422059208,2024-02-19,390000.0,3,2.50,1930,64904
8,4268200055,2024-02-09,245000.0,3,1.75,1740,11547
9,2883200160,2024-02-07,595000.0,4,2.00,2020,2849


In [10]:
# preprocess the data to be uploaded
import preprocess

df_input = pd.DataFrame(preprocess.wallaroo_json(newbatch))

[{'tensor': [4.0, 1.0, 1610.0, 2982.0, 1.5, 0.0, 0.0, 4.0, 7.0, 1610.0, 0.0, 47.587, -122.294, 1610.0, 4040.0, 99.0, 0.0, 0.0]}, {'tensor': [3.0, 1.0, 1610.0, 8579.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1010.0, 600.0, 47.4563, -122.171, 1610.0, 8579.0, 62.0, 0.0, 0.0]}, {'tensor': [4.0, 3.25, 5050.0, 20100.0, 1.5, 0.0, 2.0, 3.0, 11.0, 4750.0, 300.0, 47.6312, -122.223, 3890.0, 20060.0, 42.0, 1.0, 26.0]}, {'tensor': [2.0, 1.0, 1040.0, 7538.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1040.0, 0.0, 47.7107, -122.352, 1440.0, 7530.0, 85.0, 0.0, 0.0]}, {'tensor': [4.0, 2.5, 2210.0, 49375.0, 1.0, 0.0, 0.0, 3.0, 8.0, 2210.0, 0.0, 47.3828, -122.071, 2670.0, 49385.0, 27.0, 0.0, 0.0]}, {'tensor': [3.0, 1.75, 2390.0, 6000.0, 1.0, 0.0, 0.0, 3.0, 6.0, 1240.0, 1150.0, 47.5362, -122.268, 2020.0, 6000.0, 116.0, 0.0, 0.0]}, {'tensor': [5.0, 3.0, 3830.0, 8480.0, 2.0, 0.0, 1.0, 5.0, 9.0, 2630.0, 1200.0, 47.6166, -122.287, 3050.0, 7556.0, 119.0, 1.0, 89.0]}, {'tensor': [3.0, 2.5, 1930.0, 64904.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1930.0, 0.0,

In [11]:
# query = {'query': newbatch.to_json()}

result = pipeline.infer(df_input)

In [12]:
# post processing here

import postprocess

df_output = pd.DataFrame(postprocess.wallaroo_json(result))
df_output

,out.variable
0,[496046.0]
1,[270739.0]
2,[3067263.0]
3,[378917.0]
4,[496464.0]
...,...
838,[364243.0]
839,[570974.0]
840,[878860.0]
841,[537433.0]


In [13]:
# display(result)
predicted_prices = pd.DataFrame(result['out.variable'].apply(lambda x: x[0])).rename(columns={'out.variable':'prediction'})
display(predicted_prices[0:5])

,prediction
0,496046.0
1,270739.0
2,3067263.0
3,378917.0
4,496464.0


### Send Predictions to Results Staging Table

Take the predicted prices based on the inference results so they can be joined into the `house_listings` table.

Once complete, undeploy the pipeline to return the resources back to the Kubernetes environment.

In [14]:
result_table = pd.DataFrame({
    'id': newbatch['id'],
    'saleprice_estimate': predicted_prices['prediction']
})

display(result_table)

result_table.to_sql('results_table', conn, index=False, if_exists='append')

,id,saleprice_estimate
0,1695900060,496046.0
1,1432900240,270739.0
2,7960900060,3067263.0
3,6378500125,378917.0
4,2022069200,496464.0
...,...,...
838,6140100028,364243.0
839,3304300300,570974.0
840,6453550090,878860.0
841,3345700207,537433.0


In [15]:
# Display the top of the table for confirmation
pd.read_sql_query("select * from results_table limit 5", conn)

,id,saleprice_estimate
0,1695900060,496046.0
1,1432900240,270739.0
2,7960900060,3067263.0
3,6378500125,378917.0
4,2022069200,496464.0


In [16]:
conn.close()
pipeline.undeploy()

name,housing-pipe
created,2024-03-06 22:04:07.428588+00:00
last_updated,2024-03-06 22:33:25.001128+00:00
deployed,False
arch,None
accel,None
tags,
versions,"bd31aef3-3c73-4315-b11c-541034a96b32, 8ee88c49-3d09-43b6-b3a3-f1fd7010d9ba, 99204f60-a7fc-473c-881e-e17a8d512774, 8322e7d7-d5ce-417e-a626-6dc4ba327b11, 3095d26d-d7f8-4478-ac2f-92df819ceb14, 7b0e4d58-2232-4a32-8757-5152ffb0d6ef, 21952c2b-89ef-45a3-b5c0-1685af58fa10, 1e6ac00f-9e81-48c6-9956-3a5b27e44616, 0f676ed7-9cd3-4021-82d4-8b180aa4fb00, 9729b5fa-2184-4b0c-8c72-906e5871c276, 895dac3a-fbfc-462c-856c-9ad47a946bf2, a4ffd306-70f6-4668-a236-b90522f0fdcb, 374fb9d1-afd1-47bb-a57e-fd6e3134ded2, ab03935a-38e2-41c3-b05b-bb0cde7088c7, 8f2dc40b-f190-4949-9568-021649630d3f, 3af8a7e3-b817-4031-ac68-c6dd09cbad95, b0524141-5a52-4824-b959-d26892699171"
steps,housepricemodel
published,False


From here, organizations can automate this process.  Other features could be used such as data analysis using Wallaroo assays, or other features such as shadow deployments to test champion and challenger models to find which models provide the best results.